<a href="https://colab.research.google.com/github/Momentum1ab/MomentumScreener/blob/main/RRG_MLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def get_status(x, y):
    if x < 100 and y < 100:
        return 'lagging'
    elif x > 100 and y > 100:
        return 'leading'
    elif x < 100 and y > 100:
        return 'improving'
    elif x > 100 and y < 100:
        return 'weakening'

def get_color(x, y):
    if get_status(x, y) == 'lagging':
        return 'red'
    elif get_status(x, y) == 'leading':
        return 'green'
    elif get_status(x, y) == 'improving':
        return 'blue'
    elif get_status(x, y) == 'weakening':
        return 'yellow'

class RRGPlotter:
    def __init__(self, benchmark, tickers, period='2y', window=12, tail=5):
        self.benchmark = benchmark
        self.tickers = tickers[:10]  # Limit to max 10 tickers
        self.period = period
        self.window = window
        self.tail = tail
        self.load_data()
        self.calculate_rrg()
        self.create_plot()

    def load_data(self):
        self.tickers_data = yf.download(self.tickers, period=self.period, interval="1wk")['Adj Close']
        self.benchmark_data = yf.download(self.benchmark, period=self.period, interval="1wk")['Adj Close']

        print(f"Tickers data shape: {self.tickers_data.shape}")
        print(f"Benchmark data shape: {self.benchmark_data.shape}")

    def calculate_rrg(self):
        self.rs_tickers = []
        self.rsr_tickers = []
        self.rsr_roc_tickers = []
        self.rsm_tickers = []

        for ticker in self.tickers:
            print(f"Processing ticker: {ticker}")
            rs = 100 * (self.tickers_data[ticker] / self.benchmark_data)
            rsr = (100 + (rs - rs.rolling(window=self.window).mean()) / rs.rolling(window=self.window).std(ddof=0)).dropna()

            print(f"Shape of rs: {rs.shape}")
            print(f"Shape of rsr: {rsr.shape}")

            if not rsr.empty:
                rsr_roc = 100 * ((rsr / rsr.shift(1)) - 1)
                rsm = (101 + ((rsr_roc - rsr_roc.rolling(window=self.window).mean()) / rsr_roc.rolling(window=self.window).std(ddof=0))).dropna()

                print(f"Shape of rsr_roc: {rsr_roc.shape}")
                print(f"Shape of rsm: {rsm.shape}")

                self.rs_tickers.append(rs)
                self.rsr_tickers.append(rsr[rsr.index.isin(rsm.index)])
                self.rsr_roc_tickers.append(rsr_roc)
                self.rsm_tickers.append(rsm)
            else:
                print(f"Warning: Empty DataFrame for ticker {ticker}")
                self.rs_tickers.append(pd.Series())
                self.rsr_tickers.append(pd.Series())
                self.rsr_roc_tickers.append(pd.Series())
                self.rsm_tickers.append(pd.Series())

    def create_plot(self):
        self.fig, self.ax = plt.subplots(figsize=(12, 8))
        self.ax.set_title(f'RRG Indicator (Benchmark: {self.benchmark})')
        self.ax.set_xlabel('JdK RS Ratio')
        self.ax.set_ylabel('JdK RS Momentum')

        self.ax.axhline(y=100, color='k', linestyle='--')
        self.ax.axvline(x=100, color='k', linestyle='--')

        self.ax.fill_between([94, 100], [94, 94], [100, 100], color='red', alpha=0.2)
        self.ax.fill_between([100, 106], [94, 94], [100, 100], color='yellow', alpha=0.2)
        self.ax.fill_between([100, 106], [100, 100], [106, 106], color='green', alpha=0.2)
        self.ax.fill_between([94, 100], [100, 100], [106, 106], color='blue', alpha=0.2)

        self.ax.text(95, 105, 'Improving', fontsize=10)
        self.ax.text(104, 105, 'Leading', fontsize=10)
        self.ax.text(104, 95, 'Weakening', fontsize=10)
        self.ax.text(95, 95, 'Lagging', fontsize=10)

        self.ax.set_xlim(94, 106)
        self.ax.set_ylim(94, 106)

        self.scatter_plots = []
        self.line_plots = []
        self.annotations = []

        for i, ticker in enumerate(self.tickers):
            if not self.rsr_tickers[i].empty and not self.rsm_tickers[i].empty:
                color = get_color(self.rsr_tickers[i].iloc[-1], self.rsm_tickers[i].iloc[-1])
                scatter = self.ax.scatter(self.rsr_tickers[i].iloc[-1], self.rsm_tickers[i].iloc[-1], color=color, s=50)
                line, = self.ax.plot(self.rsr_tickers[i].tail(self.tail), self.rsm_tickers[i].tail(self.tail), color=color, alpha=0.5)
                annotation = self.ax.annotate(ticker, (self.rsr_tickers[i].iloc[-1], self.rsm_tickers[i].iloc[-1]), fontsize=8)

                self.scatter_plots.append(scatter)
                self.line_plots.append(line)
                self.annotations.append(annotation)
            else:
                print(f"Warning: Empty data for ticker {ticker}")

        plt.tight_layout()
        plt.show()

    def animate(self, num_frames=60):
        def update(frame):
            for i, ticker in enumerate(self.tickers):
                if not self.rsr_tickers[i].empty and not self.rsm_tickers[i].empty:
                    end_idx = min(frame + 1, len(self.rsr_tickers[i]))
                    color = get_color(self.rsr_tickers[i].iloc[end_idx-1], self.rsm_tickers[i].iloc[end_idx-1])

                    self.scatter_plots[i].set_offsets(np.c_[self.rsr_tickers[i].iloc[end_idx-1], self.rsm_tickers[i].iloc[end_idx-1]])
                    self.scatter_plots[i].set_color(color)

                    self.line_plots[i].set_data(self.rsr_tickers[i].iloc[max(0, end_idx-self.tail):end_idx],
                                                self.rsm_tickers[i].iloc[max(0, end_idx-self.tail):end_idx])
                    self.line_plots[i].set_color(color)

                    self.annotations[i].set_position((self.rsr_tickers[i].iloc[end_idx-1], self.rsm_tickers[i].iloc[end_idx-1]))

            return self.scatter_plots + self.line_plots + self.annotations

        anim = animation.FuncAnimation(self.fig, update, frames=num_frames, interval=100, blit=True)
        return HTML(anim.to_jshtml())

def get_user_input():
    benchmark = input("Enter the benchmark index ticker (e.g., ^NSEI for Nifty 50): ")
    tickers = input("Enter up to 10 tickers separated by commas (e.g., ^CNXFMCG,^CNXIT,GOLDBEES.NS): ").split(',')
    tickers = [ticker.strip() for ticker in tickers[:10]]  # Limit to max 10 tickers
    period = input("Enter the period (e.g., 1y, 2y, 5y): ")
    window = int(input("Enter the window size for calculations (e.g., 12): "))
    tail = int(input("Enter the tail length for plotting (e.g., 5): "))

    return benchmark, tickers, period, window, tail

def main():
    benchmark, tickers, period, window, tail = get_user_input()
    rrg_plotter = RRGPlotter(benchmark, tickers, period, window, tail)

    # Display static plot
    rrg_plotter.create_plot()

    # Display animated plot
    HTML(rrg_plotter.animate().data)

    # Print table with ticker information
    print(f"{'Ticker':<15}{'Name':<30}{'Price':<10}{'Change (%)':<10}{'Status':<10}")
    print("-" * 75)
    for i, ticker in enumerate(tickers):
        info = yf.Ticker(ticker).info
        name = info.get('longName', 'N/A')[:30]
        price = round(rrg_plotter.tickers_data[ticker].iloc[-1], 2)
        chg = round((price - rrg_plotter.tickers_data[ticker].iloc[0]) / rrg_plotter.tickers_data[ticker].iloc[0] * 100, 2)
        status = get_status(rrg_plotter.rsr_tickers[i].iloc[-1], rrg_plotter.rsm_tickers[i].iloc[-1])
        print(f"{ticker:<15}{name:<30}{price:<10.2f}{chg:<10.2f}{status:<10}")

if __name__ == "__main__":
    main()